<h1 id="PrincipeAutoEncoder"> 1 - Word embedding example using GloVe pre-trained model from Stanford</h1>



In the following cells, we will see an example of word embedding using one of the most used pre-trained model.

the first cell permits to load the GloVe pre-train model directly form stanford website:

In [ ]:
#First, we directly load the data from the stanford webiste (822 Mb)

!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-12-14 08:29:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-14 08:29:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-14 08:29:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

The data are under the .zip format, as a consquence, we unzip the data.

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


The following cell permits to visualize the dimensions of the vectors. It contains 400,000 words represented in 100 dimensions.

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 100)

Then, we load the different representations of the 400,000 words in dimension 100.

We then are able to print the vectors corresponding to certain words and compute the distances between those words. We choose to represent "cat", "dog" and "house".

In [ ]:
import numpy as np

embedding_dict=dict()

f=open('glove.6B.100d.txt','r',encoding="utf-8")
for line in f:
  values=line.split()
  word=values[0]
  word_vec=np.asarray(values[1:],dtype=np.float32)
  embedding_dict[word]=word_vec
f.close()

np.set_printoptions(precision=2,suppress=True)
phrase="cat dog house"
words=phrase.split()
for w in words:
  print("")
  print(w)
  v=embedding_dict[w]
  first="[" + " ".join(map(str,v[0:3]))
  last= " ".join(map(str,v[97:100]))+ "]"
  print(first,end= ""); print("..." ,end=""); print(last)

liste=[]
for w in words:
  v=embedding_dict[w]
  liste.append(v)

print("\ndistance between cat and dog: " + str(np.linalg.norm(liste[0]-liste[1])))
print("distance between cat and house: " + str(np.linalg.norm(liste[0]-liste[2])))
print("distance between dog and house: " + str(np.linalg.norm(liste[1]-liste[2])))


cat
[0.23088 0.28283 0.6318...-0.14532 0.15104 -0.71493]

dog
[0.30817 0.30938 0.52803...0.12883 0.62529 -0.52086]

house
[-0.18867 -0.040943 0.58878...0.16221 1.1238 -0.4168]

distance between cat and dog: 2.6811306
distance between cat and house: 6.3005595
distance between dog and house: 6.263389


**RESULTS:**

We can see that dog and cat which are semantically close are close in terms of vector distance (2.68) when represented in 100 dimensions. At the contrary these two words are far in terms of distance with the word house which is very semanticaly different, and they are both at the same distance from this word (~6.3)


<h1 id="PrincipeAutoEncoder"> 2 - BERT pre-trained, used for multiple choice</h1>




**2.a - BERT base**

In the following cell, a pre-train BERT base model is loaded in it's version optimized for multiple choice selection. A sentence is then submited to the model : "Dogs are some of the most intelligent animals on earth."

Then, two possible choices are presented to the model which has to select the one that fit best the submitted sentence :

Choice0 = "Dogs are capable of swimming."

choice1 = "My name is Bond."

We will see that BERT has a lot of limits.

In [ ]:
!pip install transformers
import warnings
warnings.filterwarnings('ignore')
from transformers import BertTokenizer, BertForMultipleChoice
import torch
import torch.nn.functional as F 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased', return_dict=True)

prompt ="Dogs are some of the most intelligent animals on earth."

choice0 = "Dogs are capable of swimming."
choice1 = "My name is Bond."

labels = torch.tensor(0).unsqueeze(0) 
encoding = tokenizer([[prompt, prompt], [choice0, choice1]], return_tensors='pt', padding=True)
outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels)
loss = outputs.loss
logits = outputs.logits

     |████████████████████████████████| 1.4MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 21.0MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=21b26674300805050089a2f2f54f25103ef1cff9ff3b832014a914de18f50b83
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [ ]:
F.softmax(logits)
print(F.softmax(logits))

tensor([[0.6674, 0.3326]], grad_fn=<SoftmaxBackward>)


**RESULTS:**

The probability for choice0 to be the sentence that fit best the submitted sentence is the first number in the tensor output.

The second number is the probability for choice1 to best fit the submitted sentence.

This example, can clearly show the limits of BERT. In effect, while "Dogs are capable of swimming." is clearly closer to the submitted sentence than "My name is Bond.", the model struggles to associate to this sentence a high percentage of chances for being the choice it has to select.

In addition to that, if the code is repeated many times we will see that the percentages changes from one try to another. The average probability for "Dogs are the capable of swimming." to be the correct answer is just slightly above 50% which is juste above randomness.

The fact that we only considered BERT base (and not it's larger version) should be taken in account. We therefore, are going to try with BERT large.

**2.b - BERT Large**

In [ ]:
!pip install transformers
import warnings
warnings.filterwarnings('ignore')
from transformers import BertTokenizer, BertForMultipleChoice
import torch
import torch.nn.functional as F 

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForMultipleChoice.from_pretrained('bert-large-uncased', return_dict=True)

prompt ="Dogs are some of the most intelligent animals on earth."

choice0 = "Dogs are capable of swimming."
choice1 = "My name is Bond."

labels = torch.tensor(0).unsqueeze(0) 
encoding = tokenizer([[prompt, prompt], [choice0, choice1]], return_tensors='pt', padding=True)
outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels)
loss = outputs.loss
logits = outputs.logits

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-large-uncased and are new

In [ ]:
F.softmax(logits)

tensor([[0.5988, 0.4012]], grad_fn=<SoftmaxBackward>)

**RESULTS:**

Now that we used BERT Large the results improved slightly in average.
However the results are not totally satisfying, it shows some limits of BERT.

<h1 id="PrincipeAutoEncoder"> 3 - BERT fine-tuned for QA tasks</h1>

Finally, the following cells will permit to see how to use BERT fine tuned on QA task with the SQUAD dataset.

We ask to our model to find the answer to "Why is chocolate bad for my dog?". In order to do so we give the following corpus (from "hills_pet") "Though chocolate is a tasty treat for humans, for dogs it is poisonous. Chocolate’s toxic component is theobromine. Dogs process theobromine very slowly, which allows for it to build up in their system to toxic levels. Different types of chocolate have different amounts of theobromine, but chocolate should always be out of reach of dogs."

In [ ]:
#Loading all that is necesary to use BERT for QA

!pip install transformers
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [ ]:
# First we initialize the tokenizer and load the pre-trained and fine tuned model

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    
#we write a question and then give a corpus (from "hills_pet") containing the answer to the question.
question, text = "Why is chocolate bad for my dog?", "Though chocolate is a tasty treat for humans, for dogs it is poisonous. Chocolate’s toxic component is theobromine. Dogs process theobromine very slowly, which allows for it to build up in their system to toxic levels. Different types of chocolate have different amounts of theobromine, but chocolate should always be out of reach of dogs."
input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"

input_ids = tokenizer.encode(input_text)
token_type_ids = [0 if i <= input_ids.index(102) else 1
    for i in range(len(input_ids))]

#we generate the model
mod=model(torch.tensor([input_ids]), \
    token_type_ids=torch.tensor([token_type_ids]))

all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(' '.join(all_tokens[torch.argmax(mod.start_logits) : torch.argmax(mod.end_logits)+1]))


it is poisonous


**RESULTS:**

Our first try using BERT-large fine tuned on SQUAD is a huge succes, the model is already capable on a complex example of QA to identify an answer. The given response here, is a short answer consisting of three words that can be found in the corpus and they best fit the question "it is poisonous".

